In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv('/content/drive/MyDrive/hour_dataset.csv', encoding='EUC-KR')

In [31]:
df.head()

,지점,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),...,운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,112,2023-01-01 00:00,3.0,NaN,4.6,230,62,4.7,-3.5,1020.8,...,NaN,NaN,872,NaN,40.0,-0.6,-0.1,-0.2,0.5,1.5
1,112,2023-01-01 01:00,3.2,NaN,5.1,250,64,4.9,-2.9,1020.5,...,NaN,15.0,793,NaN,40.0,-0.6,-0.1,-0.1,0.5,1.5
2,112,2023-01-01 02:00,2.8,NaN,4.1,270,70,5.2,-2.1,1020.4,...,NaN,12.0,618,NaN,40.0,-1.0,-0.1,-0.1,0.5,1.6
3,112,2023-01-01 03:00,2.3,NaN,2.8,320,71,5.1,-2.4,1020.6,...,Sc,10.0,545,NaN,40.0,-0.8,0.0,-0.1,0.5,1.5
4,112,2023-01-01 04:00,1.3,NaN,3.3,340,73,4.9,-3.0,1020.7,...,Sc,10.0,588,NaN,40.0,-0.9,-0.1,-0.2,0.5,1.5


In [32]:
df.keys()

Index(['지점', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)',
       '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)',
       '일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)', '전운량(10분위)', '중하층운량(10분위)',
       '운형(운형약어)', '최저운고(100m )', '시정(10m)', '지면상태(지면상태코드)', '현상번호(국내식)',
       '지면온도(°C)', '5cm 지중온도(°C)', '10cm 지중온도(°C)', '20cm 지중온도(°C)',
       '30cm 지중온도(°C)'],
      dtype='object')

In [33]:
df.shape

(8760, 27)

필요한 것들
- 기온
- 강수량
- 풍속
- 습도
- 현지기압
- 전운량
> 나머지는 지우자

In [34]:
labelsToRemove = ['지점', '일시', '풍향(16방위)',
'증기압(hPa)', '이슬점온도(°C)', '해면기압(hPa)', '일조(hr)',
'일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)', '중하층운량(10분위)',
'운형(운형약어)', '최저운고(100m )', '시정(10m)', '지면상태(지면상태코드)', '현상번호(국내식)',
'지면온도(°C)', '5cm 지중온도(°C)', '10cm 지중온도(°C)', '20cm 지중온도(°C)',
'30cm 지중온도(°C)']

In [35]:
for label in labelsToRemove:
	df = df.drop(label, axis=1)

In [36]:
df.head()

,기온(°C),강수량(mm),풍속(m/s),습도(%),현지기압(hPa),전운량(10분위)
0,3.0,NaN,4.6,62,1020.8,8
1,3.2,NaN,5.1,64,1020.5,7
2,2.8,NaN,4.1,70,1020.4,7
3,2.3,NaN,2.8,71,1020.6,8
4,1.3,NaN,3.3,73,1020.7,8


단위까지 붙어있으면 접근하기 귀찮으니 변경하자

In [37]:
df = df.rename(
	columns = {
		"기온(°C)" : "기온",
		"강수량(mm)" : "강수량",
		"풍속(m/s)" : "풍속",
		"습도(%)" : "습도",
		"현지기압(hPa)" : "기압",
		"전운량(10분위)" : "운량"
	}
)
df.head()

,기온,강수량,풍속,습도,기압,운량
0,3.0,NaN,4.6,62,1020.8,8
1,3.2,NaN,5.1,64,1020.5,7
2,2.8,NaN,4.1,70,1020.4,7
3,2.3,NaN,2.8,71,1020.6,8
4,1.3,NaN,3.3,73,1020.7,8


결측값(NaN) 값을 체크해보자

In [38]:
for key in df.keys():
	print(sum(df[key].isna()))

0
7792
0
0
0
0


다행히 강수량에만 결측값이 있다.
> 그러나 강수량이 NaN 값인 것은 그냥 비가 안 온 거지 계측을 못했다는 것이 아니므로 0으로 채운다

In [39]:
df = df.fillna(0)
df.head()

,기온,강수량,풍속,습도,기압,운량
0,3.0,0.0,4.6,62,1020.8,8
1,3.2,0.0,5.1,64,1020.5,7
2,2.8,0.0,4.1,70,1020.4,7
3,2.3,0.0,2.8,71,1020.6,8
4,1.3,0.0,3.3,73,1020.7,8


이제 적절히 데이터의 범위를 변경하기 위해 최소 최댓값을 체크한다

In [40]:
for key in df.keys():
	print(key, min(df[key]), max(df[key]))

기온 -16.7 34.6
강수량 0.0 30.4
풍속 0.0 11.3
습도 10 99
기압 984.1 1030.1
운량 0 10


- 기온 | -20 ~ 40 => 0 ~ 1.
- 강수량 | 0 => 0 / >0 => 1
- 풍속 | 0 ~ 12. => 0 ~ 1.
- 습도 | 0 ~ 100 => 0 ~ 1.
- 운량 | 0 ~ 3 => 0 / 4 ~ 7 => 1

In [42]:
df['기온'] = (df['기온'] + 20) / 60
df['강수량'] = df['강수량'].apply(lambda x: 1 if x > 0 else 0)
df['풍속'] = df['풍속'] / 12
df['습도'] = df['습도'] / 100
df['운량'] = df['운량'].replace([0,1,2,3], 0).replace([4,5,6,7,8,9,10], 1)

In [43]:
df.head()

,기온,강수량,풍속,습도,기압,운량
0,0.383333,0,0.383333,0.62,1020.8,1
1,0.386667,0,0.425000,0.64,1020.5,1
2,0.380000,0,0.341667,0.70,1020.4,1
3,0.371667,0,0.233333,0.71,1020.6,1
4,0.355000,0,0.275000,0.73,1020.7,1


In [44]:
dataY = np.array(df['강수량'])
dataY

array([0, 0, 0, ..., 0, 0, 0])

In [45]:
dataX = np.array(list(zip(df['기온'], df['운량'], df['풍속'], df['습도'], df['기압'])))
dataX

array([[3.83333333e-01, 1.00000000e+00, 3.83333333e-01, 6.20000000e-01,
        1.02080000e+03],
       [3.86666667e-01, 1.00000000e+00, 4.25000000e-01, 6.40000000e-01,
        1.02050000e+03],
       [3.80000000e-01, 1.00000000e+00, 3.41666667e-01, 7.00000000e-01,
        1.02040000e+03],
       ...,
       [3.61666667e-01, 0.00000000e+00, 2.00000000e-01, 8.70000000e-01,
        1.01630000e+03],
       [3.50000000e-01, 0.00000000e+00, 7.50000000e-02, 8.90000000e-01,
        1.01690000e+03],
       [3.45000000e-01, 0.00000000e+00, 2.16666667e-01, 9.00000000e-01,
        1.01730000e+03]])

In [46]:
dataX.shape, dataY.shape

((8760, 5), (8760,))

이제 학습 고고씽

In [51]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size=0.2)


In [52]:
import tensorflow as tf

In [53]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(64, activation='relu'),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(256, activation='relu'),
	tf.keras.layers.Dense(512, activation='relu'),
	tf.keras.layers.Dense(1, activation='sigmoid'),
])
# model = tf.keras.Sequential([
#   tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(5,))  # 입력 특성 5개, 출력 1개 (운량 예측)
# ])

In [54]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [57]:
model.fit(trainX, trainY, epochs=1, validation_data=(testX, testY))

219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9320 - loss: 0.1797 - val_accuracy: 0.9424 - val_loss: 0.1406


In [58]:
model.evaluate(testX, testY)
#loss, accuracy

55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9424 - loss: 0.1470


[0.1406157910823822, 0.9423515796661377]

In [59]:
import datetime
now = datetime.datetime.now()
model.save(f'/content/drive/MyDrive/sdgsmodels/weather_{now.strftime("%Y%m%d_%H%M%S")}.h5')